In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from pathlib import Path
import pymc as pm
import arviz as az
import multiprocessing as mp
from estival.wrappers import pymc as epm
from estival.sampling import tools as esamp
from estival.wrappers import nevergrad as eng
from estival.utils.parallel import map_parallel
import nevergrad as ng

from tbdynamics.constants import BURN_IN, OPTI_DRAWS
from autumn.infrastructure.remote import springboard
from tbdynamics.calib_utils import get_bcm
import pandas as pd
from estival.utils.sample import SampleTypes


In [ ]:
params = {
    # "contact_rate": 0.007255277552812773,
    # "start_population_size": 2200000,
    # "rr_infection_latent": 0.4435087078206438,
    # "rr_infection_recovered": 0.3726339126072788,
    # "progression_multiplier": 1.71734470040371,
    # "seed_time": 1810.0,
    # "seed_num": 70.0,
    # "seed_duration": 10.0,
    # "smear_positive_death_rate": 0.3789083132417181,
    # "smear_negative_death_rate": 0.023640460692223537,
    # "smear_positive_self_recovery": 0.2064856493052769,
    # "smear_negative_self_recovery": 0.11108908197816095,
    # "screening_scaleup_shape": 0.1289863504422179,
    # "screening_inflection_time": 1993.8078541156067,
    # "screening_end_asymp": 0.5820504987396568,
    # "detection_reduction": 0.7542772690107872,
}
bcm = get_bcm(params)

In [ ]:
def calibrate(out_path, bcm, draws, tune):
    def optimize_ng_with_idx(item):
        idx, sample = item
        opt = eng.optimize_model(bcm, budget=100, opt_class=ng.optimizers.TwoPointsDE, suggested = sample, num_workers=4)
        rec= opt.minimize(100)
        return idx, rec.value[1]

    lhs_samples = bcm.sample.lhs(16, ci=0.67)
    lhs_lle = esamp.likelihood_extras_for_samples(lhs_samples, bcm)
    lhs_sorted = lhs_lle.sort_values("loglikelihood", ascending=False)
    opt_samples_idx = map_parallel(optimize_ng_with_idx, lhs_sorted.iterrows())
    best_opt_samps = bcm.sample.convert(opt_samples_idx)
    init_samps = best_opt_samps.convert(SampleTypes.LIST_OF_DICTS)[0:8]
    n_chains = 8
    n_samples = 200
    with pm.Model() as pm_model:
        variables = epm.use_model(bcm)
        idata_raw = pm.sample(
            step=[pm.DEMetropolisZ(variables)],
            draws=draws,
            tune=tune,
            discard_tuned_samples=False,
            chains=n_chains,
            progressbar=True,
            initvals=init_samps,
        )
    idata_raw.to_netcdf(str(out_path / "calib_full_out.nc"))

    burnt_idata = idata_raw.sel(draw=np.s_[BURN_IN:])
    idata_extract = az.extract(burnt_idata, num_samples=n_samples)
    bcm.sample.convert(idata_extract).to_hdf5(out_path / "calib_extract_out.h5")

    spaghetti_res = esamp.model_results_for_samples(idata_extract, bcm)
    spaghetti_res.results.to_hdf(str(out_path / "results.hdf"), "spaghetti")

    like_df = esamp.likelihood_extras_for_idata(idata_raw, bcm)
    like_df.to_hdf(str(out_path / "results.hdf"), "likelihood")


def run_calibration(bridge: springboard.task.TaskBridge, bcm, draws, tune):
    import multiprocessing as mp
    mp.set_start_method("forkserver")
    idata_raw = calibrate(bridge.out_path, bcm, draws, tune)
    bridge.logger.info("Calibration complete")

In [ ]:
OUT_PATH = Path.cwd() / 'runs/r1004'
draws= 100000
tune = 25000
calibrate(OUT_PATH,bcm, draws, tune)

In [ ]:
# idata_raw

In [ ]:
draws = 100000
tune = 20000

commands = [
    'git clone --branch tb-covid https://github.com/longbui/tbdynamics.git',
    'pip install -e ./tbdynamics',
]

mspec = springboard.EC2MachineSpec(6, 2, 'compute')
run_str = f'd{int(draws / 1000)}k-t{int(tune / 1000)}k-b{int(BURN_IN / 1000)}k'
tspec_args = {'bcm': bcm,'draws': draws, 'tune': tune}
tspec = springboard.TaskSpec(run_calibration, tspec_args)
run_path =  springboard.launch.get_autumn_project_run_path('tbdynamics', 's10043_calibration', run_str)
runner = springboard.launch.launch_synced_autumn_task(tspec, mspec, run_path, branch=None, extra_commands=commands)


In [ ]:
# print(runner.tail())

In [ ]:
# from autumn.infrastructure.remote import springboard
# rts = springboard.task.RemoteTaskStore()
# rts.cd('projects/tbdynamics/s10043_calibration')
# rts.ls()

# mt.get_log("crash")

In [ ]:
# mt = rts.get_managed_task('2024-04-11T2322-d100k-t20k-b20k')

In [ ]:
# print(mt.get_log('crash'))

In [ ]:
# print(runner.get_log("crash"))